In [41]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import warnings
warnings.simplefilter("ignore")
import altair as alt
from vega_datasets import data

In [42]:
price_df = pd.read_csv('dataset/price.csv', sep = '\t')

In [43]:
cpi_df = pd.read_csv('dataset/cpi.csv', sep = '\t')

In [44]:
cpi_chart = alt.Chart(cpi_df).mark_line(point=True).encode(
    x=alt.X('Year:O',axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('Avg:Q',title='CPI'),
    color=alt.value("#FFAA00")
)

In [45]:
price_chart = alt.Chart(price_df).mark_line(point=True).encode(
    x=alt.X('Year:O',axis=alt.Axis(labelAngle=-45)),
    y='mean(price):Q',
    color = 'state:N'
)


In [46]:
(price_chart+cpi_chart).resolve_scale(
    y='independent'
).properties(
    title='Mean Tickect Price from 1994-2022 Season',
    width = 500,
    height=300
)

alt.LayerChart(...)

## By state

In [47]:
states_url = data.us_10m.url
state_id = data.population_engineers_hurricanes()[['state','id']]
states = alt.topo_feature(data.us_10m.url, feature = 'states')
temp_df = pd.read_csv('dataset/climdiv_state_year.csv')

In [48]:
temp_df =temp_df.rename(columns={'fips':'id'})
temp_new_df = temp_df[(temp_df.year >= 1994)]

In [49]:
selection_state=alt.selection_single(fields=['id'], init={"id":9})
colorCondition = alt.condition(selection_state,alt.value(1.0), alt.value(0.1))

In [50]:
year_slider = alt.binding_range(min=1994, max=2022, step=1, name="Year")
slider_selection = alt.selection_single(bind=year_slider, fields=['Year'])

In [51]:
price_state_df = pd.merge(price_df,state_id,left_on='state', right_on='state',how='left')

In [52]:
price_state_mean_df = price_state_df.groupby(['state','Year','id']).agg(
    {'price': 'mean'}
).reset_index()

In [53]:
scale = alt.Scale(
    domain=[20, 120],
    type='linear'
)

In [54]:
background = alt.Chart(price_state_mean_df).transform_lookup(
    lookup='id',
    from_=alt.LookupData(states,'id',fields=['geometry','type'])
).mark_geoshape(
    stroke='black', strokeWidth=1,
).encode(
    color=alt.Color('price', scale=scale)
).add_selection(
    slider_selection,selection_state
).transform_filter(
    slider_selection
).project(
    type='albersUsa'
).properties(
    width = 300,
    height= 500
)

In [55]:
price_selection = alt.Chart(price_state_df).transform_lookup(
    lookup='id',
    from_=alt.LookupData(states,'id',fields=['geometry','type'])
).transform_filter(
    selection_state
).mark_line(point=True).encode(
    x=alt.X('Year:O',axis=alt.Axis(labelAngle=-45),title=None),
    y=alt.Y('mean(price):Q',title='Ticket Price ($)'),
)

In [56]:
temp_selection = alt.Chart(temp_new_df).transform_lookup(
    lookup='id',
    from_=alt.LookupData(states,'id',fields=['geometry','type'])
).transform_filter(
    selection_state
).mark_line(point=True).encode(
    x=alt.X('year:O',axis=alt.Axis(labelAngle=-45),title=None),
    y=alt.Y('temp:Q',title='Temperature (℉)',scale=alt.Scale(domain=[40, 55])),
    color=alt.value("#FFAA00")
)


In [57]:
price_cpi_chart = (price_selection + cpi_chart ).resolve_scale(
    y='independent'
).properties(
    title=alt.TitleParams(text='Tickect Price vs. CPI', fontSize=12),
    width = 300,
    height = 200
)

In [58]:
price_temp_chart = (price_selection + temp_selection).resolve_scale(
    y='independent',
).properties(
    title=alt.TitleParams(text='Tickect Price vs. Temperature', fontSize=12),
    width = 300,
    height= 200
)

In [59]:
price_change_chart = (background | (price_cpi_chart & price_temp_chart)).configure_title(fontSize=30).properties(
        title = 'Ticket Price From 1994-2022'
    )

In [60]:
price_change_chart

alt.HConcatChart(...)